In [2]:
import pandas as pd

df = pd.read_csv("./data_cts_intentional_homicide.csv")
df = df.drop_duplicates()
df = df.drop(columns=['Iso3_code', 'Region', 'Subregion', 'Dimension', 'Indicator','Source', 'Category' ])
drop_pop = df[df['Unit of measurement'] == 'Rate per 100,000 population'].index
df = df.drop(drop_pop)

In [3]:
print(df.isnull().any())

Country                False
Sex                    False
Age                    False
Year                   False
Unit of measurement    False
VALUE                  False
dtype: bool


In [4]:
#source issue
df = (
    df[pd.to_numeric(df['VALUE'], errors='coerce').dropna().apply(float.is_integer)]
    .reset_index(drop=True)
)
df

,Country,Sex,Age,Year,Unit of measurement,VALUE
0,Armenia,Male,Total,2013,Counts,35.0
1,Switzerland,Male,Total,2013,Counts,28.0
2,Colombia,Male,Total,2013,Counts,15053.0
3,Czechia,Male,Total,2013,Counts,69.0
4,Germany,Male,Total,2013,Counts,455.0
...,...,...,...,...,...,...
59719,Australia and New Zealand,Male,Total,2013,Counts,185.0
59720,Australia and New Zealand,Male,Total,2015,Counts,183.0
59721,Australia and New Zealand,Male,Total,2016,Counts,175.0
59722,Western Europe,Male,Total,2016,Counts,1206.0


In [5]:
df['Year'] = pd.to_datetime(df.Year, format='%Y')
print(df.dtypes)

Country                        object
Sex                            object
Age                            object
Year                   datetime64[ns]
Unit of measurement            object
VALUE                         float64
dtype: object


In [6]:
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [7]:
lack_data = {country: len(data) for country, data in df.groupby('Country') if len(data) < 100}
lack_data

{'Afghanistan': 13,
 'American Samoa': 27,
 'Angola': 4,
 'Anguilla': 37,
 'Aruba': 51,
 'Australia and New Zealand': 32,
 'Bahrain': 54,
 'Bangladesh': 19,
 'Botswana': 21,
 'British Virgin Islands': 37,
 'Brunei Darussalam': 26,
 'Burundi': 13,
 'Cabo Verde': 96,
 'Cambodia': 20,
 'Cameroon': 22,
 'Cayman Islands': 69,
 'China': 32,
 'Cuba': 84,
 'Curaçao': 21,
 'Djibouti': 3,
 'Eastern Europe': 10,
 'Eritrea': 1,
 'Eswatini': 44,
 'Ethiopia': 1,
 'Faroe Islands': 9,
 'Fiji': 41,
 'French Guiana': 6,
 'French Polynesia': 4,
 'Ghana': 24,
 'Gibraltar': 2,
 'Greenland': 25,
 'Guadeloupe': 14,
 'Guam': 13,
 'Guinea-Bissau': 2,
 'Haiti': 37,
 'Indonesia': 13,
 'Iran (Islamic Republic of)': 21,
 'Iraq': 13,
 'Iraq (Central Iraq)': 69,
 'Iraq (Kurdistan Region)': 8,
 'Isle of Man': 6,
 'Israel': 88,
 'Kiribati': 16,
 'Kuwait': 45,
 'Kyrgyzstan': 15,
 'Lebanon': 50,
 'Lesotho': 7,
 'Liberia': 6,
 'Madagascar': 23,
 'Malawi': 13,
 'Malaysia': 34,
 'Maldives': 73,
 'Marshall Islands': 3,
 'Ma

In [8]:
countries_to_drop = list(lack_data.keys())

# Drop rows from the DataFrame for these countries
df = df[~df['Country'].isin(countries_to_drop)]
unique_countries = df['Country'].unique()
unique_countries

array(['Armenia', 'Switzerland', 'Colombia', 'Czechia', 'Germany',
       'Finland', 'Guatemala', 'Honduras', 'Hungary', 'Iceland', 'Italy',
       'Japan', 'Sri Lanka', 'Lithuania', 'Mongolia', 'Norway', 'Serbia',
       'Slovakia', 'Slovenia', 'Türkiye', 'Antigua and Barbuda',
       'Austria', 'Belgium', 'Belize', 'Bolivia (Plurinational State of)',
       'Bhutan', 'Denmark', 'France', 'Greece',
       'China, Macao Special Administrative Region', 'Russian Federation',
       'Uzbekistan', 'Albania', 'Azerbaijan', 'Barbados', 'Chile',
       'Spain', 'Grenada', 'Guyana', 'Croatia', 'Liechtenstein', 'Latvia',
       'Montenegro', 'Panama', 'Dominica', 'Jordan',
       'Saint Kitts and Nevis', 'Saint Lucia', 'Mexico', 'Malta', 'Oman',
       'Trinidad and Tobago', 'Bulgaria', 'Bahamas', 'Canada',
       'Costa Rica', 'Dominican Republic', 'El Salvador', 'Uruguay',
       'Bosnia and Herzegovina', 'Saint Vincent and the Grenadines',
       'Ecuador', 'Holy See', 'Morocco', 'Mauritius'

In [9]:
df

,Country,Sex,Age,Year,Unit of measurement,VALUE
0,Armenia,Male,Total,2013-01-01,Counts,35.0
1,Switzerland,Male,Total,2013-01-01,Counts,28.0
2,Colombia,Male,Total,2013-01-01,Counts,15053.0
3,Czechia,Male,Total,2013-01-01,Counts,69.0
4,Germany,Male,Total,2013-01-01,Counts,455.0
...,...,...,...,...,...,...
59648,Hungary,Total,Total,2023-01-01,Counts,1.0
59649,Jordan,Total,Total,2023-01-01,Counts,0.0
59650,Latvia,Total,Total,2023-01-01,Counts,0.0
59651,Puerto Rico,Total,Total,2023-01-01,Counts,2.0


In [10]:
df = df.drop(columns=['Unit of measurement' ])
#handle the Sex column
unique_genders = df['Sex'].unique()
unique_genders

array(['Male', 'Female', 'Total'], dtype=object)

In [11]:
# Generate dummy variables for 'Sex'
df = pd.get_dummies(df, columns=['Sex'], prefix='Gender')

# Convert dummy columns to integers
df = df.astype({'Gender_Female': 'int', 'Gender_Male': 'int', 'Gender_Total': 'int'})
df

,Country,Age,Year,VALUE,Gender_Female,Gender_Male,Gender_Total
0,Armenia,Total,2013-01-01,35.0,0,1,0
1,Switzerland,Total,2013-01-01,28.0,0,1,0
2,Colombia,Total,2013-01-01,15053.0,0,1,0
3,Czechia,Total,2013-01-01,69.0,0,1,0
4,Germany,Total,2013-01-01,455.0,0,1,0
...,...,...,...,...,...,...,...
59648,Hungary,Total,2023-01-01,1.0,0,0,1
59649,Jordan,Total,2023-01-01,0.0,0,0,1
59650,Latvia,Total,2023-01-01,0.0,0,0,1
59651,Puerto Rico,Total,2023-01-01,2.0,0,0,1


In [12]:
#handle age column
unique_age =  df['Age'].unique()
unique_age

array(['Total', 'Unknown', '0-9', '10 -14', '15 -17', '18-19', '20-24',
       '25-29', '30-44', '45-59', '60 and older'], dtype=object)

In [13]:
#'Total' '0 to 17 years' '18 years or over'
# Define a mapping dictionary for age groups
age_group_mapping = {
    'Unknown': 'Total',
    '0-9': '0 to 17 years',
    '10 -14': '0 to 17 years',
    '15 -17': '0 to 17 years',
    '18-19': '18 years or over',
    '20-24': '18 years or over',
    '25-29': '18 years or over',
    '30-44': '18 years or over',
    '45-59': '18 years or over',
    '60 and older': '18 years or over'
}

# Replace values using the mapping
df['Age'] = df['Age'].replace(age_group_mapping)
df

,Country,Age,Year,VALUE,Gender_Female,Gender_Male,Gender_Total
0,Armenia,Total,2013-01-01,35.0,0,1,0
1,Switzerland,Total,2013-01-01,28.0,0,1,0
2,Colombia,Total,2013-01-01,15053.0,0,1,0
3,Czechia,Total,2013-01-01,69.0,0,1,0
4,Germany,Total,2013-01-01,455.0,0,1,0
...,...,...,...,...,...,...,...
59648,Hungary,Total,2023-01-01,1.0,0,0,1
59649,Jordan,Total,2023-01-01,0.0,0,0,1
59650,Latvia,Total,2023-01-01,0.0,0,0,1
59651,Puerto Rico,Total,2023-01-01,2.0,0,0,1


In [14]:
# Generate dummy variables for 'Sex'
df = pd.get_dummies(df, columns=['Age'], prefix='Age')
print(df.columns)

Index(['Country', 'Year', 'VALUE', 'Gender_Female', 'Gender_Male',
       'Gender_Total', 'Age_0 to 17 years', 'Age_18 years or over',
       'Age_Total'],
      dtype='object')


In [15]:
df = df.astype({'Age_Total': 'int', 'Age_0 to 17 years': 'int', 'Age_18 years or over': 'int'})
df = df.rename(columns={'Age_0 to 17 years': 'Age_Under18', 'Age_18 years or over': 'Age_Over18'})
df['VALUE'] = df['VALUE'].astype(int)
cols = [col for col in df.columns if col != 'VALUE']
cols.append('VALUE')
df = df[cols]
df

,Country,Year,Gender_Female,Gender_Male,Gender_Total,Age_Under18,Age_Over18,Age_Total,VALUE
0,Armenia,2013-01-01,0,1,0,0,0,1,35
1,Switzerland,2013-01-01,0,1,0,0,0,1,28
2,Colombia,2013-01-01,0,1,0,0,0,1,15053
3,Czechia,2013-01-01,0,1,0,0,0,1,69
4,Germany,2013-01-01,0,1,0,0,0,1,455
...,...,...,...,...,...,...,...,...,...
59648,Hungary,2023-01-01,0,0,1,0,0,1,1
59649,Jordan,2023-01-01,0,0,1,0,0,1,0
59650,Latvia,2023-01-01,0,0,1,0,0,1,0
59651,Puerto Rico,2023-01-01,0,0,1,0,0,1,2


In [15]:
df.to_csv('D3_data.csv', index=False)

In [1]:
import pandas as pd
df1 = pd.read_csv("./D1_data.csv")
df2 = pd.read_csv("./D2_data.csv")
df3 = pd.read_csv("./D3_data.csv")
df_combined = pd.concat([df1, df2, df3], ignore_index=True)
df_combined = df_combined.sort_values(by=['Country', 'Year'])
df_combined = df_combined.reset_index(drop=True)
df_combined.to_csv('combined_sorted_crime_data.csv', index=False)